In [6]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [12]:
#Datenpfad und Verzeichnisse
DATA_DIR = "data"
RAW_DATA_FILE = os.path.join(DATA_DIR, "all_energy_statistics.csv")
OUTPUT_DIR = "output"

# Sicherstellen, dass das Output-Verzeichnis existiert
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 3. Rohdaten laden

try:
    df = pd.read_csv(RAW_DATA_FILE)
    print("Daten erfolgreich geladen. Vorschau:")
    display(df.head())
except FileNotFoundError:
    print(f"Fehler: Datei {RAW_DATA_FILE} nicht gefunden.")

Daten erfolgreich geladen. Vorschau:


,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


In [ ]:
# Überblick über die Struktur der Rohdaten

print(f"Anzahl Zeilen und Spalten: {df.shape}")
print("\nSpaltennamen:")
print(df.columns.tolist())

print("\nFehlende Werte pro Spalte:")
print(df.isnull().sum())

print("\nDatentypen:")
print(df.dtypes)

Anzahl Zeilen und Spalten: (1189482, 7)

Spaltennamen:
['country_or_area', 'commodity_transaction', 'year', 'unit', 'quantity', 'quantity_footnotes', 'category']

Fehlende Werte pro Spalte:
country_or_area                0
commodity_transaction          0
year                           0
unit                           0
quantity                       0
quantity_footnotes       1025536
category                       0
dtype: int64

Datentypen:
country_or_area           object
commodity_transaction     object
year                       int64
unit                      object
quantity                 float64
quantity_footnotes       float64
category                  object
dtype: object


In [13]:
# Zeilen mit vorhandenen (nicht NaN) Footnotes anzeigen
df_footnotes = df[df["quantity_footnotes"].notna()]

# Vorschau der ersten Zeilen mit Footnotes
display(df_footnotes.head())

# Übersicht: Wieviele Zeilen haben eine Footnote?
print(f"Anzahl Zeilen mit Footnotes: {df_footnotes.shape[0]}")

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
622,Colombia,Additives and Oxygenates - Production,2014,"Metric tons, thousand",34.53,1.0,additives_and_oxygenates
1369,Colombia,Additives and Oxygenates - Total energy supply,2014,"Metric tons, thousand",34.53,1.0,additives_and_oxygenates
1886,Colombia,Additives and Oxygenates - transfers and recyc...,2014,"Metric tons, thousand",34.53,1.0,additives_and_oxygenates
2250,Jordan,Additives and Oxygenates - Transformation,2014,"Metric tons, thousand",103.00,1.0,additives_and_oxygenates
2251,Jordan,Additives and Oxygenates - Transformation,2013,"Metric tons, thousand",101.00,1.0,additives_and_oxygenates


Anzahl Zeilen mit Footnotes: 163946


## Cleaning of commodity

In [16]:
# Maximal 100 Zeilen in der Ausgabe anzeigen, damit wir viele Einträge auf einmal sehen können
pd.set_option('display.max_rows', 100)

In [17]:
# Verteilung der Hauptkategorien (nur zur Information)
display(df.category.value_counts())

category
total_electricity                                              133916
gas_oil_diesel_oil                                              97645
fuel_oil                                                        75132
natural_gas_including_lng                                       64161
liquified_petroleum_gas                                         62156
motor_gasoline                                                  53198
fuelwood                                                        52032
electricity_net_installed_capacity_of_electric_power_plants     50229
other_kerosene                                                  43466
hard_coal                                                       42307
kerosene_type_jet_fuel                                          34558
heat                                                            29656
charcoal                                                        28000
conventional_crude_oil                                          27352
other_bitum

In [18]:
# Übersicht über die seltensten commodity_transaction-Einträge
display(df.commodity_transaction.value_counts().tail(50))

commodity_transaction
Kerosene-type Jet Fuel - Transformation in CHP plants - main activity producers                       1
Patent fuel - Consumption by non-ferrous metals                                                       1
White spirit and special boiling point industrial spirits - Consumption by textile and leather        1
Biogasoline - Consumption not elsewhere specified (transport)                                         1
Paraffin waxes - Consumption by wood and wood products                                                1
Paraffin waxes - Consumption by transport equipment                                                   1
Blast Furnace Gas - Consumption by food and tobacco                                                   1
White spirit and special boiling point industrial spirits - Consumption by wood and wood products     1
White spirit and special boiling point industrial spirits - Consumption by transport equipment        1
Paraffin waxes - Consumption by paper, pul

In [19]:
# Vorschau: Wie sehen die ersten commodity_transaction-Einträge aus?
display(df.commodity_transaction.head())

0    Additives and Oxygenates - Exports
1    Additives and Oxygenates - Exports
2    Additives and Oxygenates - Exports
3    Additives and Oxygenates - Exports
4    Additives and Oxygenates - Exports
Name: commodity_transaction, dtype: object

In [20]:
# Anzahl Trennzeichen in den commodity_transaction-Strings zählen
display(df.commodity_transaction.str.count(" - | – ").value_counts())

commodity_transaction
1    1135821
2      47472
0       6189
Name: count, dtype: int64

In [21]:
# Aufteilen in drei neue Spalten: commodity, transaction_type und additional_transaction_info
split_commodities = df.commodity_transaction.str.split(" - | – ", expand=True)

# Vorschau auf die neuen Spalten
display(split_commodities.head())

,0,1,2
0,Additives and Oxygenates,Exports,None
1,Additives and Oxygenates,Exports,None
2,Additives and Oxygenates,Exports,None
3,Additives and Oxygenates,Exports,None
4,Additives and Oxygenates,Exports,None


In [22]:
# Übersicht: Inhalt der zweiten Spalte (transaction_type)
display(split_commodities[1].str.lower().value_counts())

# Übersicht: Inhalt der dritten Spalte (additional_transaction_info)
display(split_commodities[2].str.lower().value_counts())

# Übersicht: Inhalt der ersten Spalte (commodity)
display(split_commodities[0].str.lower().str.strip().value_counts())

1
total energy supply                         89806
final consumption                           73149
final energy consumption                    68195
imports                                     61151
production                                  61129
                                            ...  
consumption by lng/regasification plants        3
own use by charcoal plants                      3
own use by gasworks                             2
non energy uses                                 1
net transfers                                   1
Name: count, Length: 224, dtype: int64

2
main activity producers    18462
autoproducers              15788
electricity plants          5887
chp plants                  3974
heat plants                 1854
total                       1507
Name: count, dtype: int64

0
electricity                                                  165140
gas oil/ diesel oil                                           97645
fuel oil                                                      75132
natural gas (including lng)                                   64161
motor gasoline                                                53198
fuelwood                                                      52032
liquefied petroleum gas (lpg)                                 49078
other kerosene                                                43466
hard coal                                                     40214
kerosene-type jet fuel                                        34558
charcoal                                                      28000
other bituminous coal                                         23138
other oil products n.e.c.                                     22081
lubricants                                                    21374
conventional crude oil                        

In [23]:
# Anzeige erweitern, um mehr Werte zu sehen
pd.set_option('display.max_rows', 250)

# Kleinbuchstaben und Entfernen von Leerzeichen vorne/hinten
split_commodities[1] = split_commodities[1].str.lower().str.strip()

# Korrektur von häufigen Tippfehlern und unsauberen Schreibweisen
split_commodities[1] = split_commodities[1].str.replace("transformatin", "transformation", regex=False)
split_commodities[1] = split_commodities[1].str.replace("non energy uses", "consumption for non-energy uses", regex=False)
split_commodities[1] = split_commodities[1].str.replace(" /", "/", regex=False)
split_commodities[1] = split_commodities[1].str.replace("/ ", "/", regex=False)

# Kontrolle: Verteilung der bereinigten transaction_type
display(split_commodities[1].value_counts())

1
total energy supply                                                                    89806
final consumption                                                                      73149
final energy consumption                                                               68195
imports                                                                                61151
production                                                                             61129
consumption by other                                                                   43549
exports                                                                                36052
transformation                                                                         35725
stock changes                                                                          32816
consumption by households                                                              32367
production from refineries                                          

In [24]:
# Nochmals prüfen: Was steht in additional_transaction_info?
display(split_commodities[2].str.lower().str.strip().value_counts())

2
main activity producers    18462
autoproducers              15788
electricity plants          5887
chp plants                  3974
heat plants                 1854
total                       1507
Name: count, dtype: int64

In [25]:
# Neue Spalten umbenennen, bevor wir sie ins Haupt-DataFrame einfügen
split_commodities.columns = ["commodity", "transaction_type", "additional_transaction_info"]

# Zusammenfügen entlang der Spaltenachse (axis=1)
df = pd.concat([df, 
                split_commodities["commodity"].str.lower(),
                split_commodities["transaction_type"],
                split_commodities["additional_transaction_info"].str.lower()], 
              axis=1)

# Vorschau: DataFrame nach Zusammenführung
display(df.head())

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category,commodity,transaction_type,additional_transaction_info
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates,additives and oxygenates,exports,None
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates,additives and oxygenates,exports,None
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates,exports,None
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates,additives and oxygenates,exports,None
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates,additives and oxygenates,exports,None


## Dropping and renaming irrelevant Data

In [28]:
df.drop(columns=["commodity_transaction"], inplace=True)
df.drop(columns=["quantity_footnotes"], inplace=True)


In [29]:
unit_counts = df["unit"].value_counts()

# Ausgabe
print("Verfügbare Einheiten und deren Häufigkeit:")
display(unit_counts)

Verfügbare Einheiten und deren Häufigkeit:


unit
Metric tons,  thousand     759859
Terajoules                 178937
Kilowatt-hours, million    147741
Cubic metres, thousand      52032
Kilowatts,  thousand        50229
Metric Tons                   684
Name: count, dtype: int64

In [30]:
# Zeilen filtern, wo die Einheit "Metric Tons" ist
df_metric_tons = df[df["unit"] == "Metric Tons"]

# Vorschau
display(df_metric_tons)

print(f"Anzahl Zeilen mit Einheit 'Metric Tons': {df_metric_tons.shape[0]}")

,country_or_area,year,unit,quantity,category,commodity,transaction_type,additional_transaction_info
1177352,Argentina,2011,Metric Tons,11000.0,uranium,uranium,estimated additional reserves,None
1177353,Australia,2011,Metric Tons,558700.0,uranium,uranium,estimated additional reserves,None
1177354,Botswana,2011,Metric Tons,59100.0,uranium,uranium,estimated additional reserves,None
1177355,Brazil,2011,Metric Tons,121000.0,uranium,uranium,estimated additional reserves,None
1177356,Canada,2011,Metric Tons,192500.0,uranium,uranium,estimated additional reserves,None
...,...,...,...,...,...,...,...,...
1178031,Uzbekistan,2014,Metric Tons,59400.0,uranium,uranium,reasonably assured reserves,None
1178032,Uzbekistan,2011,Metric Tons,64300.0,uranium,uranium,reasonably assured reserves,None
1178033,Viet Nam,2011,Metric Tons,1000.0,uranium,uranium,reasonably assured reserves,None
1178034,Zambia,2011,Metric Tons,9900.0,uranium,uranium,reasonably assured reserves,None


Anzahl Zeilen mit Einheit 'Metric Tons': 684


In [31]:
# Bedingung: Nur Zeilen mit Einheit "Metric Tons"
mask_metric_tons = df["unit"] == "Metric Tons"

# Menge durch 1000 teilen
df.loc[mask_metric_tons, "quantity"] = df.loc[mask_metric_tons, "quantity"] / 1000

# Einheit umbenennen auf "Metric tons, thousand", damit alles konsistent ist
df.loc[mask_metric_tons, "unit"] = "Metric tons, thousand"

print("Anpassung abgeschlossen. Einheit und Mengen für 'Metric Tons' sind jetzt normalisiert.")

Anpassung abgeschlossen. Einheit und Mengen für 'Metric Tons' sind jetzt normalisiert.


## Bereinigtes CS Speichern und Check-Load: Gespeicherte Datei neu einlesen und prüfen


In [32]:
# Bereinigtes DataFrame als neue CSV speichern
# Neuer Pfad und Dateiname
OUTPUT_CLEANED_FILE = "/Users/yanickfischer/Documents/vsCode/MDM/mdm-project2-data-wrangling/data/energy_cleaned.csv"

# DataFrame als CSV speichern, ohne zusätzlichen Index
df.to_csv(OUTPUT_CLEANED_FILE, index=False)

print(f"Datei erfolgreich gespeichert unter: {OUTPUT_CLEANED_FILE}")

Datei erfolgreich gespeichert unter: /Users/yanickfischer/Documents/vsCode/MDM/mdm-project2-data-wrangling/data/energy_cleaned.csv


In [33]:
# Datei neu einlesen
try:
    df_check = pd.read_csv(OUTPUT_CLEANED_FILE)
    print("Datei erfolgreich neu geladen. Vorschau:")
    display(df_check.head())
    
    print("\nForm der geladenen Datei:")
    print(df_check.shape)
except FileNotFoundError:
    print(f"Fehler: Datei {OUTPUT_CLEANED_FILE} nicht gefunden.")

Datei erfolgreich neu geladen. Vorschau:


/var/folders/2k/c1ct_2b11hl7zvvtlkp0f5ww0000gn/T/ipykernel_63832/1628874401.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_check = pd.read_csv(OUTPUT_CLEANED_FILE)


,country_or_area,year,unit,quantity,category,commodity,transaction_type,additional_transaction_info
0,Austria,1996,"Metric tons, thousand",5.0,additives_and_oxygenates,additives and oxygenates,exports,NaN
1,Austria,1995,"Metric tons, thousand",17.0,additives_and_oxygenates,additives and oxygenates,exports,NaN
2,Belgium,2014,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,NaN
3,Belgium,2013,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,NaN
4,Belgium,2012,"Metric tons, thousand",35.0,additives_and_oxygenates,additives and oxygenates,exports,NaN



Form der geladenen Datei:
(1189482, 8)


## Filter for European countries

In [34]:
# Liste europäischer Länder (sorgfältig zusammengestellt)
european_countries = [
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg",
    "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland",
    "Ukraine", "United Kingdom", "Vatican City"
]

# DataFrame auf europäische Länder filtern
df_europe = df[df["country_or_area"].isin(european_countries)]

# Vorschau auf das Ergebnis
display(df_europe.head())

print(f"Anzahl Zeilen nach Filterung auf europäische Länder: {df_europe.shape[0]}")

,country_or_area,year,unit,quantity,category,commodity,transaction_type,additional_transaction_info
0,Austria,1996,"Metric tons, thousand",5.0,additives_and_oxygenates,additives and oxygenates,exports,None
1,Austria,1995,"Metric tons, thousand",17.0,additives_and_oxygenates,additives and oxygenates,exports,None
2,Belgium,2014,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,None
3,Belgium,2013,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,None
4,Belgium,2012,"Metric tons, thousand",35.0,additives_and_oxygenates,additives and oxygenates,exports,None


Anzahl Zeilen nach Filterung auf europäische Länder: 417186


In [35]:
# Alle einzigartigen Länder im DataFrame extrahieren und alphabetisch sortieren
unique_countries = sorted(df["country_or_area"].unique())

# Anzeige aller Länder
print("Liste der vorhandenen Länder im Datensatz:")
for country in unique_countries:
    print(country)

# Gesamtzahl der unterschiedlichen Länder
print(f"\nAnzahl unterschiedlicher Länder im Datensatz: {len(unique_countries)}")

Liste der vorhandenen Länder im Datensatz:
Afghanistan
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctic Fisheries
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia (Plur. State of)
Bonaire, St Eustatius, Saba
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Rep.
Chad
Chile
China
China, Hong Kong SAR
China, Macao SAR
Colombia
Commonwealth of Independent States (CIS)
Comoros
Congo
Cook Islands
Costa Rica
Croatia
Cuba
Curaçao
Cyprus
Czechia
Czechoslovakia (former)
Côte d'Ivoire
Dem. Rep. of the Congo
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Ethiopia, incl. Eritrea
Faeroe Islands
Falkland Is. (Malvinas)
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
G

In [37]:
# Mapping von alternativen zu gesuchten Ländernamen
country_name_mapping = {
    "Czechia": "Czech Republic",
    "Republic of Moldova": "Moldova",
    "T.F.Yug.Rep. Macedonia": "North Macedonia"
    # Kosovo, Monaco, San Marino, Vatican City könnten tatsächlich fehlen
}

# Ländernamen im DataFrame ersetzen
df["country_or_area"] = df["country_or_area"].replace(country_name_mapping)

print("Ländernamen angepasst. Jetzt sind alternative Bezeichnungen vereinheitlicht.")

# Liste europäischer Länder (wie vorher definiert)
european_countries = [
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg",
    "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland",
    "Ukraine", "United Kingdom", "Vatican City"
]

# Alle eindeutigen Länder im Datensatz
unique_countries = set(df["country_or_area"].unique())

# Vergleich: Welche europäischen Länder fehlen?
missing_countries = [country for country in european_countries if country not in unique_countries]

# Ausgabe
if missing_countries:
    print("Diese europäischen Länder fehlen im Datensatz:")
    for country in missing_countries:
        print(f"- {country}")
else:
    print("Alle europäischen Länder aus der Liste sind im Datensatz vorhanden!")

Ländernamen angepasst. Jetzt sind alternative Bezeichnungen vereinheitlicht.
Diese europäischen Länder fehlen im Datensatz:
- Kosovo
- Monaco
- San Marino
- Vatican City


In [41]:
# Europäische Länderliste erneut definieren (falls nicht schon vorhanden)
european_countries = [
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece",
    "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg",
    "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland",
    "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland",
    "Ukraine", "United Kingdom", "Vatican City"
]

# DataFrame auf europäische Länder filtern (nach dem Mapping)
df_europe = df[df["country_or_area"].isin(european_countries)]

# Vorschau auf das gefilterte DataFrame
display(df_europe.head())

# Anzahl Zeilen nach Filter
print(f"\nAnzahl Zeilen mit europäischen Ländern: {df_europe.shape[0]}")

# Einzigartige europäischen Länder im gefilterten Datensatz
unique_european_countries = sorted(df_europe["country_or_area"].unique())

print(f"\nAnzahl unterschiedlicher europäischer Länder im Datensatz: {len(unique_european_countries)}")
print("\nListe der im Datensatz enthaltenen europäischen Länder:")
for country in unique_european_countries:
    print(f"- {country}")

# Überprüfung, ob noch Länder fehlen
missing_countries_final = [country for country in european_countries if country not in set(unique_european_countries)]

if missing_countries_final:
    print("\nDiese europäischen Länder fehlen nach finaler Anpassung weiterhin:")
    for country in missing_countries_final:
        print(f"- {country}")
else:
    print("\nAlle europäischen Länder aus der Liste sind jetzt im Datensatz vorhanden!")

,country_or_area,year,unit,quantity,category,commodity,transaction_type,additional_transaction_info
0,Austria,1996,"Metric tons, thousand",5.0,additives_and_oxygenates,additives and oxygenates,exports,None
1,Austria,1995,"Metric tons, thousand",17.0,additives_and_oxygenates,additives and oxygenates,exports,None
2,Belgium,2014,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,None
3,Belgium,2013,"Metric tons, thousand",0.0,additives_and_oxygenates,additives and oxygenates,exports,None
4,Belgium,2012,"Metric tons, thousand",35.0,additives_and_oxygenates,additives and oxygenates,exports,None



Anzahl Zeilen mit europäischen Ländern: 446601

Anzahl unterschiedlicher europäischer Länder im Datensatz: 41

Liste der im Datensatz enthaltenen europäischen Länder:
- Albania
- Andorra
- Austria
- Belarus
- Belgium
- Bosnia and Herzegovina
- Bulgaria
- Croatia
- Cyprus
- Czech Republic
- Denmark
- Estonia
- Finland
- France
- Germany
- Greece
- Hungary
- Iceland
- Ireland
- Italy
- Latvia
- Liechtenstein
- Lithuania
- Luxembourg
- Malta
- Moldova
- Montenegro
- Netherlands
- North Macedonia
- Norway
- Poland
- Portugal
- Romania
- Serbia
- Slovakia
- Slovenia
- Spain
- Sweden
- Switzerland
- Ukraine
- United Kingdom

Diese europäischen Länder fehlen nach finaler Anpassung weiterhin:
- Kosovo
- Monaco
- San Marino
- Vatican City


In [42]:
# Neuer Speicherpfad und Dateiname
OUTPUT_EUROPE_FILE = "/Users/yanickfischer/Documents/vsCode/MDM/mdm-project2-data-wrangling/data/energy_cleaned_europe.csv"

# DataFrame als CSV abspeichern, ohne Indexspalte
df_europe.to_csv(OUTPUT_EUROPE_FILE, index=False)

print(f"Europäischer Datensatz erfolgreich gespeichert unter: {OUTPUT_EUROPE_FILE}")

Europäischer Datensatz erfolgreich gespeichert unter: /Users/yanickfischer/Documents/vsCode/MDM/mdm-project2-data-wrangling/data/energy_cleaned_europe.csv


In [43]:
# Überblick über numerische Werte (Mittelwert, Standardabweichung, Min, Max usw.)
display(df_europe.describe())

# Überblick über die Häufigkeit der Commodities (Energieträger)
print("\nVerteilung der Energieträger (commodity):")
display(df_europe["commodity"].value_counts())

# Überblick über die Häufigkeit der Transaktionstypen (transaction_type)
print("\nVerteilung der Transaktionstypen (transaction_type):")
display(df_europe["transaction_type"].value_counts())

# Überblick über die verwendeten Einheiten
print("\nVerwendete Einheiten (unit):")
display(df_europe["unit"].value_counts())

,year,quantity
count,446601.000000,4.466010e+05
mean,2003.131966,3.597335e+04
std,7.062069,1.353547e+06
min,1990.000000,-4.772630e+05
25%,1997.000000,1.300000e+01
50%,2004.000000,1.920000e+02
75%,2009.000000,2.224000e+03
max,2014.000000,2.303000e+08



Verteilung der Energieträger (commodity):


commodity
electricity                                                  46497
gas oil/ diesel oil                                          30459
natural gas (including lng)                                  28035
fuel oil                                                     26256
hard coal                                                    17420
fuelwood                                                     16147
heat                                                         15373
liquefied petroleum gas (lpg)                                14500
other bituminous coal                                        14336
motor gasoline                                               12754
brown coal                                                   10599
other oil products n.e.c.                                     9015
kerosene-type jet fuel                                        8741
other kerosene                                                8676
petroleum coke                                      


Verteilung der Transaktionstypen (transaction_type):


transaction_type
total energy supply                                                                    26387
final consumption                                                                      21858
final energy consumption                                                               20287
imports                                                                                19493
production                                                                             18456
stock changes                                                                          14757
exports                                                                                14201
transformation                                                                         13587
consumption by other                                                                   13430
consumption by households                                                               9980
transformation in electricity, chp and heat plants   


Verwendete Einheiten (unit):


unit
Metric tons,  thousand     272581
Terajoules                  99032
Kilowatt-hours, million     45117
Cubic metres, thousand      16147
Kilowatts,  thousand        13499
Metric tons, thousand         225
Name: count, dtype: int64